# IMDb 감성 분석: 사전 훈련된 GloVe 임베딩 활용

이 노트북은 사전 훈련된 단어 임베딩(GloVe)을 사용하여 IMDb 영화 리뷰 감성 분석 모델의 성능을 개선하는 방법을 다룹니다. 모델이 처음부터 단어 임베딩을 학습하는 대신, 대규모 텍스트 코퍼스로부터 학습된 지식을 활용합니다.

주요 단계는 다음과 같습니다:
1. 데이터 다운로드 및 전처리
2. 텍스트 벡터화 및 어휘 사전 구축
3. GloVe 임베딩 파일 로드
4. 우리 데이터셋의 어휘 사전에 맞는 임베딩 행렬 생성
5. 사전 훈련된 임베딩을 사용하는 Keras 모델 구축 및 훈련

## 1. 라이브러리 임포트

프로젝트에 필요한 라이브러리를 임포트합니다.

In [ ]:
import requests
import subprocess
import os
import pathlib
import shutil
import random
import numpy as np
import tensorflow as tf
import keras
from keras import models, layers
from tensorflow.keras.layers import TextVectorization

## 2. 데이터 준비

이전 노트북과 동일하게 IMDb 데이터셋을 다운로드하고, 압축을 해제한 후, 훈련 및 검증 세트로 구성합니다. 이 과정은 데이터가 준비되지 않았을 경우에만 실행됩니다.

In [ ]:
def download():
    """지정된 URL에서 aclImdb_v1.tar.gz 파일을 다운로드합니다."""
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    file_name = "aclImdb_v1.tar.gz"
    if not os.path.exists(file_name):
        print(f"{file_name} 다운로드를 시작합니다...")
        response = requests.get(url, stream=True)
        with open(file_name, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print("다운로드 완료!")
    else:
        print(f"{file_name}이(가) 이미 존재합니다.")

def release():
    """tar.gz 파일의 압축을 해제합니다."""
    if not os.path.exists("aclImdb"):
        print("압축 해제를 시작합니다...")
        subprocess.run(["tar", "-xvzf", "aclImdb_v1.tar.gz"], shell=True)
        print("압축 해제 완료!")
        if os.path.exists("aclImdb/train/unsup"):
            shutil.rmtree("aclImdb/train/unsup")
            print("불필요한 unsup 디렉토리를 삭제했습니다.")
    else:
        print("aclImdb 디렉토리가 이미 존재합니다.")

def labeling():
    """훈련 세트의 일부를 검증 세트로 분리합니다."""
    base_dir = pathlib.Path("aclImdb")
    val_dir = base_dir / "val"
    train_dir = base_dir / "train"
    if not val_dir.exists():
        print("검증 세트 생성을 시작합니다...")
        for category in ("neg", "pos"):
            os.makedirs(val_dir / category)
            files = os.listdir(train_dir / category)
            random.Random(1337).shuffle(files)
            num_val_samples = int(0.2 * len(files))
            val_files = files[-num_val_samples:]
            for fname in val_files:
                shutil.move(train_dir / category / fname, val_dir / category / fname)
        print("검증 세트 생성 완료!")
    else:
        print("val 디렉토리가 이미 존재합니다.")

# 데이터 준비 함수 실행
download()
release()
labeling()

## 3. 데이터셋 로드 및 텍스트 벡터화

데이터를 `tf.data.Dataset`으로 로드하고, `TextVectorization` 층을 사용하여 텍스트를 정수 시퀀스로 변환할 준비를 합니다. 이 과정에서 우리 데이터셋에 맞는 어휘 사전을 구축합니다.

In [ ]:
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size=batch_size)

max_length = 600
max_tokens = 20000

text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

# 어휘 사전 구축
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

## 4. 사전 훈련된 GloVe 임베딩 로드

Stanford의 GloVe 임베딩 파일을 파싱하여 단어를 해당 벡터 표현에 매핑하는 인덱스를 구축합니다. `glove.6B.100d.txt` 파일은 100차원 임베딩 벡터를 포함하고 있습니다.

In [ ]:
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file, encoding="utf-8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"총 {len(embeddings_index)}개의 단어 벡터를 찾았습니다.")

## 5. 임베딩 행렬 생성

`TextVectorization`을 통해 만든 어휘 사전을 기반으로, Keras `Embedding` 층에 주입할 수 있는 임베딩 행렬을 생성합니다. 이 행렬의 각 행 `i`는 어휘 사전에서 인덱스가 `i`인 단어에 해당하는 `100`차원 GloVe 벡터를 포함합니다.

In [ ]:
vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_dim = 100  # GloVe 파일에 따라 차원 설정
embedding_matrix = np.zeros((max_tokens, embedding_dim))

for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # 어휘 사전에 있는 단어이고, GloVe에 존재하는 경우 해당 벡터로 채웁니다.
            embedding_matrix[i] = embedding_vector

print("임베딩 행렬이 준비되었습니다.")

## 6. 모델 구축

이제 사전 훈련된 임베딩을 사용하여 모델을 정의합니다. `Embedding` 층을 생성할 때 `embeddings_initializer`에 방금 만든 `embedding_matrix`를 전달하고, `trainable=False`로 설정하여 GloVe 가중치가 훈련 중에 업데이트되지 않도록 동결합니다.

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")

# 사전 훈련된 임베딩 행렬을 사용하여 Embedding 층 초기화
embedded = layers.Embedding(
    input_dim=max_tokens, 
    output_dim=embedding_dim, 
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,  # 임베딩 가중치 동결
    mask_zero=True
)(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.summary()

## 7. 모델 훈련 및 평가

마지막으로, 데이터셋을 벡터화하고 모델을 훈련시킨 후 테스트셋으로 최종 성능을 평가합니다.

In [ ]:
# 데이터셋에 벡터화 적용
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE)

print("\n모델 훈련 시작:")
callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

print("\n테스트셋 평가 결과:")
model.evaluate(int_test_ds)